#  Import Body Map from Infopad csv file

#### Last updated on : 3rd Oct 2020  -  Amar Jaiswal

In [1]:
es_base_url = 'http://localhost:9200'
headers = {'Content-Type': 'application/json'}

In [2]:
# auto sugestion with tab
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as datetime

import requests
import json

import os
os.getcwd()

'/lhome/amardj/ISM/python_code/es_scripts'

In [4]:
body_main_file = '/lhome/amardj/ISM/python_code/es_scripts/resources/backend_supportprim_data/2020-10/body_map/patient_body_map_201008.csv'

In [5]:
raw_csv = pd.read_csv( body_main_file, delimiter=';')

In [6]:
raw_csv.head()

,Brukernavn,Journal ID,Fødselsår,Forside 0 0,Forside 0 1,Forside 0 2,Forside 0 3,Forside 0 4,Forside 0 5,Forside 0 6,...,Bakside 7 5,Bakside 7 6,Bakside 7 7,Bakside 7 8,Bakside 7 9,Bakside 7 10,Bakside 7 11,Bakside 7 12,Bakside 7 13,Bakside 7 14
0,support1,150680,1980,,,,,,,,...,,,,,,,,,,
1,support1,1506801,1980,,,,,,,,...,,,,,,,,,,
2,support1,1506802,1980,,,,,,,,...,,,,,,,,,,
3,support1,1506803,1980,,,,,,,,...,,,,,,,,,,
4,support1,1506804,1980,,,,,,,,...,,,,,,,,,,


In [7]:
raw = raw_csv.rename(columns={'Fødselsår': 'birthYear', 'Brukernavn':'clinicId', 'Journal ID':'journalId'})

In [8]:
exclude_patients_from_clinic = raw[
    (raw.clinicId != 'Fredrik Granviken') 
    & (raw.clinicId != 'Anita Formo Bones')
    & (raw.clinicId != 'Ingebrigt Meisingset')
    & (raw.clinicId != 'Testinstitutt')
    & (raw.clinicId != 'support32')
    & (raw.clinicId != 'support3')
] #"Testinstitutt", "support32","support3"

In [9]:
exclude_patients_from_clinic.shape

(180, 243)

In [10]:
exclude_patients_from_clinic.head()

,clinicId,journalId,birthYear,Forside 0 0,Forside 0 1,Forside 0 2,Forside 0 3,Forside 0 4,Forside 0 5,Forside 0 6,...,Bakside 7 5,Bakside 7 6,Bakside 7 7,Bakside 7 8,Bakside 7 9,Bakside 7 10,Bakside 7 11,Bakside 7 12,Bakside 7 13,Bakside 7 14
0,support1,150680,1980,,,,,,,,...,,,,,,,,,,
1,support1,1506801,1980,,,,,,,,...,,,,,,,,,,
2,support1,1506802,1980,,,,,,,,...,,,,,,,,,,
3,support1,1506803,1980,,,,,,,,...,,,,,,,,,,
4,support1,1506804,1980,,,,,,,,...,,,,,,,,,,


In [11]:
es = []

for index, df in exclude_patients_from_clinic.iterrows():
    #print(index)
    
    patient = {}
    patient_details = {}
    body_map = {}
    
    df_1_row = df.replace(' ',np.nan).dropna(axis=0,how="all")
    
    patient.update({'patientId':
                    df_1_row[0].replace(' ','_').lower()
                    +'_'+str(df_1_row[1])
                    +'_'+str(df_1_row[2])})
    
    front = []
    back = []

    for val in df_1_row.index[3:].tolist():
        tokens = str(val).split(' ')

        if tokens[0] == 'Forside':     
            front.append({ 'x':int(tokens[1]), 'y':int(tokens[2])})

        if tokens[0] == 'Bakside':
            back.append({ 'x':int(tokens[1]), 'y':int(tokens[2])})
    
    body_map.update({'front':front})
    body_map.update({'back':back})
    
    patient_details.update({'body_map':body_map})
    patient.update({'patientDetails':patient_details})

    es.append(patient)
    
es;

In [12]:
final_json = json.dumps(es)

In [13]:
with open('body_map.json', 'w', encoding='utf-8') as f:
    json.dump(es, f, ensure_ascii=False, indent=0)

In [14]:
for patient in es:
    print(patient.get('patientId'))

support1_150680_1980
support1_1506801_1980
support1_1506802_1980
support1_1506803_1980
support1_1506804_1980
support1_1001_1980
support1_1002_1980
support1_1003_1980
support1_1006_1980
support1_1007_1980
support1_1013_1980
gaute_alstad_sunde_9145_1975
vidar_guttormsen_17504_1973
astrid_holmen_39164_1999
adam_saso_39151_1997
adam_saso_39178_1999
adam_saso_39185_1993
alette_bade_rygh_1803_1999
vidar_guttormsen_30160_1992
support17_1148_1970
helge_pedersen_1_1963
helge_pedersen_2_1965
support18_11730_1956
adam_saso_39224_1969
astrid_holmen_8853_1968
astrid_holmen_16449_1959
support16_13053_1957
marianne_bakka_31894_1956
fredrik_paulsberg_31903_1972
support23_1_1982
support23_2_1965
support27_1_1970
fredrik_paulsberg_31910_1968
vidar_guttormsen_31911_1995
vidar_guttormsen_31912_1971
support23_3_1950
support7_5655_1992
support26_1001_1950
adam_saso_38888_1998
support17_1152_1991
marianne_bakka_31909_1995
vidar_guttormsen_31959_1969
alette_bade_rygh_1809_1984
support21_17564_1972
alette_bade

In [15]:
initial_patient_list = [
 'adam_saso_38888_1998',
 'adam_saso_39151_1997',
 'adam_saso_39178_1999',
 'adam_saso_39185_1993',
 'adam_saso_39442_1989',
 'alette_bade_rygh_1808_1969',
 'alette_bade_rygh_1810_1989',
 'astrid_holmen_8853_1968',
 'astrid_holmen_39465_1967',
 'fredrik_paulsberg_31903_1972',
 'fredrik_paulsberg_31910_1968',
 'fredrik_paulsberg_31992_1961',
 'fredrik_paulsberg_32035_1969',
 'gaute_alstad_sunde_2748_1996',
 'gaute_alstad_sunde_8017_1973',
 'gaute_alstad_sunde_8092_1982',
 'gaute_alstad_sunde_9145_1975',
 'gaute_alstad_sunde_9327_1955',
 'helge_pedersen_1_1963',
 'helge_pedersen_2_1965',
 'helge_pedersen_5_1983',
 'helge_pedersen_7_1956',
 'helge_pedersen_8_1967',
 'helge_pedersen_9_1966',
 'helge_pedersen_10_1964',
 'helge_pedersen_11_1966',
 'helge_pedersen_12_1950',
 'marianne_bakka_31894_1956',
 'marianne_bakka_31909_1995',
 'marianne_bakka_32177_1961',
 'vidar_guttormsen_11785_1956',
 'vidar_guttormsen_17504_1973',
 'vidar_guttormsen_26296_1962',
 'vidar_guttormsen_30160_1992',
 'vidar_guttormsen_31911_1995',
 'vidar_guttormsen_31912_1971',
 'vidar_guttormsen_31959_1969',
 'vidar_guttormsen_32015_1965',
 'vidar_guttormsen_32110_1959',
 'vidar_guttormsen_32111_1998',
 'support15_6245_1969',
 'support19_5191_1968',
 'support23_3_1950',
 'support24_101_1984',
 'support24_102_1978',
 'support26_1001_1950',
 'support28_2000_1973',
 'support28_2002_1981',
 'support31_32359_1979',
 'support7_3816_1984',
 'support7_5655_1992'
]

In [16]:
#initial_patient_list.str.lower()
initial_patient_list = [x.lower() for x in initial_patient_list]
initial_patient_list

['adam_saso_38888_1998',
 'adam_saso_39151_1997',
 'adam_saso_39178_1999',
 'adam_saso_39185_1993',
 'adam_saso_39442_1989',
 'alette_bade_rygh_1808_1969',
 'alette_bade_rygh_1810_1989',
 'astrid_holmen_8853_1968',
 'astrid_holmen_39465_1967',
 'fredrik_paulsberg_31903_1972',
 'fredrik_paulsberg_31910_1968',
 'fredrik_paulsberg_31992_1961',
 'fredrik_paulsberg_32035_1969',
 'gaute_alstad_sunde_2748_1996',
 'gaute_alstad_sunde_8017_1973',
 'gaute_alstad_sunde_8092_1982',
 'gaute_alstad_sunde_9145_1975',
 'gaute_alstad_sunde_9327_1955',
 'helge_pedersen_1_1963',
 'helge_pedersen_2_1965',
 'helge_pedersen_5_1983',
 'helge_pedersen_7_1956',
 'helge_pedersen_8_1967',
 'helge_pedersen_9_1966',
 'helge_pedersen_10_1964',
 'helge_pedersen_11_1966',
 'helge_pedersen_12_1950',
 'marianne_bakka_31894_1956',
 'marianne_bakka_31909_1995',
 'marianne_bakka_32177_1961',
 'vidar_guttormsen_11785_1956',
 'vidar_guttormsen_17504_1973',
 'vidar_guttormsen_26296_1962',
 'vidar_guttormsen_30160_1992',
 'vi

In [31]:
init_es = []

for index, df in exclude_patients_from_clinic.iterrows():
    #print(index)
    

    
    df_1_row = df.replace(' ',np.nan).dropna(axis=0,how="all")
    
    patient_id = df_1_row[0].replace(' ','_').lower() + '_'+str(df_1_row[1]) +'_'+str(df_1_row[2])
            
    if patient_id in initial_patient_list:
        
        print(patient_id)
        
        patient = {}
        patient_details = {}
        body_map = {}

        patient.update({'patientId': patient_id}) 

        front = []
        back = []

        for val in df_1_row.index[3:].tolist():
            tokens = str(val).split(' ')

            if tokens[0] == 'Forside':     
                front.append({ 'x':int(tokens[1]), 'y':int(tokens[2])})

            if tokens[0] == 'Bakside':
                back.append({ 'x':int(tokens[1]), 'y':int(tokens[2])})

        body_map.update({'front':front})
        body_map.update({'back':back})

        patient_details.update({'body_map':body_map})
        patient.update({'patientDetails':patient_details})

        init_es.append(patient)
    
init_es;

support1_150680_1980
support1_1506801_1980
support1_1506802_1980
support1_1506803_1980
support1_1506804_1980
support1_1001_1980
support1_1002_1980
support1_1003_1980
support1_1006_1980
support1_1007_1980
support1_1013_1980
gaute_alstad_sunde_9145_1975
vidar_guttormsen_17504_1973
astrid_holmen_39164_1999
adam_saso_39151_1997
adam_saso_39178_1999
adam_saso_39185_1993
alette_bade_rygh_1803_1999
vidar_guttormsen_30160_1992
support17_1148_1970
helge_pedersen_1_1963
helge_pedersen_2_1965
support18_11730_1956
adam_saso_39224_1969
astrid_holmen_8853_1968
astrid_holmen_16449_1959
support16_13053_1957
marianne_bakka_31894_1956
fredrik_paulsberg_31903_1972
support23_1_1982
support23_2_1965
support27_1_1970
fredrik_paulsberg_31910_1968
vidar_guttormsen_31911_1995
vidar_guttormsen_31912_1971
support23_3_1950
support7_5655_1992
support26_1001_1950
adam_saso_38888_1998
support17_1152_1991
marianne_bakka_31909_1995
vidar_guttormsen_31959_1969
alette_bade_rygh_1809_1984
support21_17564_1972
alette_bade

In [18]:
with open('initial_patient_body_map.json', 'w', encoding='utf-8') as f:
    json.dump(init_es, f, ensure_ascii=False, indent=0)

In [19]:
init_es[:5]

[{'patientId': 'gaute_alstad_sunde_9145_1975',
  'patientDetails': {'body_map': {'front': [],
    'back': [{'x': 3, 'y': 12},
     {'x': 3, 'y': 13},
     {'x': 3, 'y': 14},
     {'x': 4, 'y': 2},
     {'x': 4, 'y': 13},
     {'x': 5, 'y': 13},
     {'x': 6, 'y': 5},
     {'x': 6, 'y': 8}]}}},
 {'patientId': 'vidar_guttormsen_17504_1973',
  'patientDetails': {'body_map': {'front': [{'x': 2, 'y': 7}],
    'back': [{'x': 5, 'y': 7}]}}},
 {'patientId': 'adam_saso_39151_1997',
  'patientDetails': {'body_map': {'front': [{'x': 3, 'y': 10}], 'back': []}}},
 {'patientId': 'adam_saso_39178_1999',
  'patientDetails': {'body_map': {'front': [],
    'back': [{'x': 3, 'y': 3}, {'x': 4, 'y': 3}, {'x': 4, 'y': 4}]}}},
 {'patientId': 'adam_saso_39185_1993',
  'patientDetails': {'body_map': {'front': [{'x': 3, 'y': 2},
     {'x': 4, 'y': 2}],
    'back': [{'x': 3, 'y': 2},
     {'x': 4, 'y': 2},
     {'x': 4, 'y': 3},
     {'x': 4, 'y': 4}]}}}]

# 
# 
# 

# Elasticsearch Operations

## Code for getting a patient by patientId from the ES

# 
## Set the elasticsearch (es) index name
## 

In [20]:
es_index = 'manual_patients'

# 

In [21]:
def get_all_docs(es_index=es_index):
    response = requests.post(es_base_url+'/'+es_index+'/_search?pretty', headers=headers, data='{}')
    return response.json()

In [22]:
def get_doc_by_pid(pid, es_index=es_index,):
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.post(es_base_url+'/'+es_index+'/_search?pretty', headers=headers, data=data)
    return response.json()

In [23]:
result_json = get_doc_by_pid('vidar_guttormsen_30160_1992');

In [24]:
if result_json.get('abc', {}).get('xyz', None) is None:
    print('None')

None


In [25]:
a = None

if a is None:
    pass
elif a == False: # elif not a
    pass
elif a == True: # elif a
    pass



if not a:
    pass
elif a:
    pass


In [26]:
def get_id_for_pid( pid, es_index=es_index):
    params = (('filter_path', 'hits.hits._source'))
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.get(f'{es_base_url}/{es_index}/_search?filter_path=hits.hits._id', headers=headers, data=data)
    response_json = response.json()
    return response_json.get('hits', {}).get('hits', [{}])[0].get('_id')

In [27]:
get_id_for_pid('vidar_guttormsen_30160_1992')

'NxdYDXUBPCTLibsE0F5U'

# 

# 

## Code for getting only the list of patient ids from the ES

In [28]:
def update_es_doc_for_id( _id, json_doc, es_index=es_index):
    upsert_doc = json.dumps({'doc': json_doc, 'doc_as_upsert': True})
    response = requests.post(f'{es_base_url}/{es_index}/_update/{_id}?pretty', headers=headers, data=upsert_doc)
    return response.json()

In [29]:
def upadate_es_index(body_maps_list=init_es, es_index=es_index):
    for body_map in body_maps_list:
        pid = body_map.get('patientId')
        print(pid)
        _id = get_id_for_pid( pid, es_index)
        print(_id)
        
        if _id is None:
            print('_id was None')
        else :
            print('Upserting the pid:',pid, ' and _id: ', _id)
            update_es_doc_for_id(_id, body_map, es_index)

In [30]:
upadate_es_index( init_es, 'manual_patients')

gaute_alstad_sunde_9145_1975
FRdYDXUBPCTLibsEz14w
Upserting the pid: gaute_alstad_sunde_9145_1975  and _id:  FRdYDXUBPCTLibsEz14w
vidar_guttormsen_17504_1973
MxdYDXUBPCTLibsE0F4x
Upserting the pid: vidar_guttormsen_17504_1973  and _id:  MxdYDXUBPCTLibsE0F4x
adam_saso_39151_1997
9xdYDXUBPCTLibsEzl0I
Upserting the pid: adam_saso_39151_1997  and _id:  9xdYDXUBPCTLibsEzl0I
adam_saso_39178_1999
-RdYDXUBPCTLibsEzl02
Upserting the pid: adam_saso_39178_1999  and _id:  -RdYDXUBPCTLibsEzl02
adam_saso_39185_1993
-xdYDXUBPCTLibsEzl1M
Upserting the pid: adam_saso_39185_1993  and _id:  -xdYDXUBPCTLibsEzl1M
vidar_guttormsen_30160_1992
NxdYDXUBPCTLibsE0F5U
Upserting the pid: vidar_guttormsen_30160_1992  and _id:  NxdYDXUBPCTLibsE0F5U
helge_pedersen_1_1963
GRdYDXUBPCTLibsEz15T
Upserting the pid: helge_pedersen_1_1963  and _id:  GRdYDXUBPCTLibsEz15T
helge_pedersen_2_1965
GxdYDXUBPCTLibsEz15m
Upserting the pid: helge_pedersen_2_1965  and _id:  GxdYDXUBPCTLibsEz15m
astrid_holmen_8853_1968
AxdYDXUBPCTLibsE

# 
# 
# 
# .....................................................................................................................................................................
# 
# 
# Code only for testing
# 

In [214]:
#get_docs('test')
print(json.dumps(get_docs('test/_id/1'), indent=3))

{
   "error": "no handler found for uri [/test/_id/1/_search?pretty] and method [POST]"
}


In [215]:
data = '{  "query": {    "match": {      "patientId": "Adam_Saso_38888_1998"    }  }}'
response = requests.post(es_base_url+'/manual_patients/_search', headers=headers, data=data)
patient_details = response.json();

In [216]:
pd.DataFrame(patient_details)

,took,timed_out,_shards,hits
failed,12,False,0.0,NaN
hits,12,False,NaN,"[{'_index': 'manual_patients', '_type': '_doc'..."
max_score,12,False,NaN,3.03495
skipped,12,False,0.0,NaN
successful,12,False,1.0,NaN
total,12,False,1.0,"{'value': 1, 'relation': 'eq'}"


In [217]:
import requests

params = (('filter_path', 'hits.hits._source'))
data = '{  "query": {    "match": {      "patientId": "Adam_Saso_38888_1998"    }  }}'
response = requests.get('http://localhost:9200/manual_patients/_search?filter_path=hits.hits._id', headers=headers, data=data)
response.json()

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('http://localhost:9200/index/type/_search?filter_path=hits.hits._source')

{'hits': {'hits': [{'_id': 'wjPUeHQBDYD9qlG-Jfxj'}]}}

In [116]:
response.json().get('hits', {}).get('hits', [{}])[0].get('_id')

NameError: name 'response' is not defined

In [241]:
data = '{"aggs" : {"result_from_es" : {"terms" : { "field" : "patientId.keyword", "size":100}}},"size" : 0}';

response = requests.post(es_base_url+'/manual_patients/_search?pretty', headers=headers, data=data)

raw_json = response.json()

patient_list = pd.DataFrame(raw_json.get('aggregations').get('result_from_es').get('buckets')).key.sort_values( ascending=True).tolist()

patient_list;

In [242]:
test_data = '{"doc": { "name": "Jane Doe" }}'
test_rsp = requests.post(es_base_url+'/test/_update/1?pretty', headers=headers, data=test_data)
test_json = test_rsp.json()

In [243]:
test_json

{'_index': 'test',
 '_type': '_doc',
 '_id': '1',
 '_version': 8,
 'result': 'noop',
 '_shards': {'total': 0, 'successful': 0, 'failed': 0},
 '_seq_no': 9,
 '_primary_term': 1}

In [244]:
data

'{"aggs" : {"result_from_es" : {"terms" : { "field" : "patientId.keyword", "size":100}}},"size" : 0}'

In [247]:
init_es[1].get('patientId')

'Adam_Saso_39151_1997'

In [248]:
get_id_for_pid('Adam_Saso_39151_1997')

'wzPUeHQBDYD9qlG-Jfxj'

In [249]:
for i in [1]:
    pid = init_es[i].get('patientId')
    _id = get_id_for_pid(pid)
    print(_id)

wzPUeHQBDYD9qlG-Jfxj


In [170]:
init_es[0].get('patientId')

'Vidar_Guttormsen_17504_1973'

In [254]:
upsert_doc = {'doc': json_doc_0, 'doc_as_upsert': True}
json.dumps(upsert_doc)

'{"doc": {"patientId": "Adam_Saso_39151_1997", "patientDetails": {"body_map": {"front": [{"x": 3, "y": 10}], "back": []}}}, "doc_as_upsert": true}'

In [188]:
json_doc_0

{'patientId': 'Vidar_Guttormsen_17504_1973',
 'patientDetails': {'body_map': {'front': [{'x': 2, 'y': 7}],
   'back': [{'x': 5, 'y': 7}]}}}

In [193]:
upsert_doc = {
    'doc': json_doc_0,
    'doc_as_upsert': True,
}

In [196]:
json.dumps(upsert_doc)

'{"doc": {"patientId": "Vidar_Guttormsen_17504_1973", "patientDetails": {"body_map": {"front": [{"x": 2, "y": 7}], "back": [{"x": 5, "y": 7}]}}}, "doc_as_upsert": true}'